In [170]:
import cv2 # python opencv for image processing
from ultralytics import YOLO # yolo v8 for object detection
from deep_sort_realtime.deepsort_tracker import DeepSort # deepsort for object tracking
import numpy as np
import sys
import colorsys
import os
import datetime

In [148]:
arguments = sys.argv[1:]

for arg in arguments:
    print(arg)
    
mode = sys.argv[1]
if mode == 'video':
  input_filename = sys.argv[2]
elif mode == 'benchmark':
  test_case_name = sys.argv[2]
  
  
# ------------ test here -------------
# 1. camera
# 2. video 'input_filename'
# 3. benchmark 'test_case_name'






current_path = os.getcwd()
test_case_name = 'CarScale'
test_case_path = current_path + '/benchmark/' + test_case_name + '/img'

IndentationError: expected an indented block (4243425384.py, line 10)

In [171]:
mode = 'benchmark'
test_case_name = 'CarScale'

In [177]:
mode = 'camera'

In [190]:
mode = 'video'
input_filename = 'input.mp4'

In [191]:
def VideoWriter(video_reader, output_filename):
    frame_width = int(video_reader.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(video_reader.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(video_reader.get(cv2.CAP_PROP_FPS))
    fourcc = cv2.VideoWriter_fourcc(*'MP4V')
    
    video_writer = cv2.VideoWriter(output_filename, fourcc, fps, (frame_width, frame_height))
    return video_writer

In [193]:
detector = YOLO("yolov8n.pt")
# detector.train(data='coco.yaml', epochs=100, imgsz=640) # Train the model
tracker = DeepSort(max_age=30)

color_size = 20 # number of objects in the frame
colors = [list(np.random.random(size=3) * 256) for _ in range(color_size)] # hold different rgb colors for each object
MIN_CONFIDENCE = 0.6

current_path = os.getcwd()
test_case_path = current_path + '/benchmark/' + test_case_name + '/img'

if mode == 'video':
  video_reader = cv2.VideoCapture(current_path + '/video/'  + input_filename)
elif mode == 'benchmark':
  video_reader = cv2.VideoCapture(test_case_path + "/%04d.jpg", cv2.CAP_IMAGES)
else:
  video_reader = cv2.VideoCapture(0)
  
video_writer = VideoWriter(video_reader, current_path + "/output/output.mp4")
  
# video_reader = cv2.VideoCapture(0) # <--- CAMERA_OPTION
# video_reader = cv2.VideoCapture("input.mp4") # <--- VIDEO_OPTION
# video_reader = cv2.VideoCapture(path_name + "/%04d.jpg", cv2.CAP_IMAGES) #<--- BENCHMARK_OPTION

frame_counter = 0
with open(current_path + '/output/output.txt', 'w') as file:
  while True:
    start_time = datetime.datetime.now()

    ### --- video reading: video -> frame --- ###
    retval, frame = video_reader.read()
    if not retval: break

    ### --- dectecting: frame -> list of ([left,top,w,h], confidence, detection_class) --- ###
    detections = detector(frame)[0].boxes.data.tolist()
    results = []

    # modify the data structure in detections
    for i in range(len(detections)):
      dectection = detections[i]

      confidence = float(dectection[4])
      if confidence < MIN_CONFIDENCE: continue

      left, top, right, bottom = \
        int(dectection[0]), int(dectection[1]), int(dectection[2]), int(dectection[3])
      w = right - left
      h = bottom - top

      detection_class = int(dectection[5])
      results.append(([left,top,w,h], confidence, detection_class))

    ### --- tracking: list of ([left,top,w,h], confidence, detection_class), frame -> tracks (i.e. track_id, ltrb) --- ###
    tracks = tracker.update_tracks(results, frame=frame)
    for track in tracks:
      if not track.is_confirmed(): continue
      # get track id
      track_id = int(track.track_id)
      # get bounding box
      ltrb = track.to_ltrb()
      xmin, ymin, xmax, ymax = int(ltrb[0]), int(ltrb[1]), int(ltrb[2]), int(ltrb[3])
      # draw track id and bounding box
      cv2.rectangle(frame, (xmin, ymin), (xmax, ymax), colors[track_id % color_size], 2)
      cv2.rectangle(frame, (xmin - 1, ymin - 30), (xmin + 31, ymin), colors[track_id % color_size], -1)
      cv2.putText(frame, str(track_id), (xmin + 5, ymin - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)

#       print("----------------------------")
#       print("frame: ")
#       print(frame_counter)
#       print("track id: ")
#       print(track_id)
#       print("bouding box (left, top, weight, height): ")
#       print(xmin, ymin, xmax - xmin, ymax - ymin)
#       print("----------------------------")
      file.write(f"{frame_counter}  {track_id}  {xmin}  {ymin}  {xmax - xmin}  {ymax - ymin}\n")

    ### --- output: display and store output frame withtracking result --- ###
    end_time = datetime.datetime.now()
    fps = f"FPS: {1 / (end_time - start_time).total_seconds():.2f}"
    cv2.putText(frame, fps, (10, 25), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 5)
    cv2.imshow("Output", frame)

    video_writer.write(frame) # <--- VIDEO_OPTION
  
    ### --- controller: quit using `q` button --- ###
    if cv2.waitKey(1) == ord("q"): break
      
    frame_counter += 1
    
# release all components
video_reader.release()
video_writer.release()

cv2.waitKey(1)
cv2.destroyAllWindows()
cv2.waitKey(1)

OpenCV: FFMPEG: tag 0x5634504d/'MP4V' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'

0: 384x640 21 cars, 1 bus, 2 trucks, 51.0ms
Speed: 3.8ms preprocess, 51.0ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 21 cars, 1 bus, 2 trucks, 61.0ms
Speed: 3.3ms preprocess, 61.0ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 22 cars, 1 bus, 2 trucks, 60.4ms
Speed: 3.2ms preprocess, 60.4ms inference, 2.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 21 cars, 1 bus, 2 trucks, 71.9ms
Speed: 5.9ms preprocess, 71.9ms inference, 2.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 22 cars, 1 bus, 2 trucks, 63.7ms
Speed: 3.7ms preprocess, 63.7ms inference, 2.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 25 cars, 1 bus, 2 trucks, 59.4ms
Speed: 3.1ms preprocess, 59.4ms inference, 2.0ms postprocess per image at shape


0: 384x640 21 cars, 2 buss, 1 truck, 49.5ms
Speed: 6.2ms preprocess, 49.5ms inference, 2.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 19 cars, 1 bus, 1 truck, 47.4ms
Speed: 3.7ms preprocess, 47.4ms inference, 2.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 18 cars, 1 bus, 1 truck, 44.3ms
Speed: 4.5ms preprocess, 44.3ms inference, 2.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 19 cars, 1 bus, 1 truck, 49.1ms
Speed: 6.7ms preprocess, 49.1ms inference, 2.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 19 cars, 1 bus, 1 truck, 53.7ms
Speed: 6.1ms preprocess, 53.7ms inference, 2.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 19 cars, 1 bus, 2 trucks, 49.2ms
Speed: 7.8ms preprocess, 49.2ms inference, 2.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 19 cars, 1 bus, 1 truck, 46.3ms
Speed: 4.4ms preprocess, 46.3ms inference, 2.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640


0: 384x640 19 cars, 2 buss, 2 trucks, 52.5ms
Speed: 2.4ms preprocess, 52.5ms inference, 2.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 18 cars, 1 bus, 3 trucks, 52.4ms
Speed: 2.5ms preprocess, 52.4ms inference, 2.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 19 cars, 1 bus, 2 trucks, 57.7ms
Speed: 4.3ms preprocess, 57.7ms inference, 2.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 20 cars, 1 bus, 3 trucks, 49.5ms
Speed: 2.4ms preprocess, 49.5ms inference, 2.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 21 cars, 1 bus, 2 trucks, 51.2ms
Speed: 3.8ms preprocess, 51.2ms inference, 2.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 18 cars, 2 buss, 2 trucks, 53.5ms
Speed: 2.4ms preprocess, 53.5ms inference, 2.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 18 cars, 2 buss, 2 trucks, 63.6ms
Speed: 3.9ms preprocess, 63.6ms inference, 2.5ms postprocess per image at shape (1, 3, 384, 640)

0:


0: 384x640 20 cars, 1 bus, 1 truck, 54.1ms
Speed: 4.4ms preprocess, 54.1ms inference, 2.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 22 cars, 1 bus, 1 truck, 46.9ms
Speed: 2.6ms preprocess, 46.9ms inference, 2.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 22 cars, 1 bus, 1 truck, 45.7ms
Speed: 2.5ms preprocess, 45.7ms inference, 2.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 24 cars, 1 bus, 1 truck, 47.2ms
Speed: 2.5ms preprocess, 47.2ms inference, 2.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 21 cars, 1 bus, 1 truck, 55.6ms
Speed: 3.9ms preprocess, 55.6ms inference, 2.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 22 cars, 1 bus, 1 truck, 56.1ms
Speed: 3.3ms preprocess, 56.1ms inference, 2.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 24 cars, 1 bus, 1 truck, 49.7ms
Speed: 4.3ms preprocess, 49.7ms inference, 2.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2

Speed: 2.5ms preprocess, 55.7ms inference, 2.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 21 cars, 1 bus, 3 trucks, 51.1ms
Speed: 3.9ms preprocess, 51.1ms inference, 2.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 19 cars, 1 bus, 3 trucks, 46.1ms
Speed: 3.2ms preprocess, 46.1ms inference, 2.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 20 cars, 1 bus, 3 trucks, 50.1ms
Speed: 2.8ms preprocess, 50.1ms inference, 2.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 20 cars, 1 bus, 3 trucks, 53.9ms
Speed: 2.8ms preprocess, 53.9ms inference, 2.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 20 cars, 1 bus, 3 trucks, 46.7ms
Speed: 2.0ms preprocess, 46.7ms inference, 2.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 20 cars, 1 bus, 3 trucks, 47.9ms
Speed: 1.9ms preprocess, 47.9ms inference, 2.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 22 cars, 1 bus, 4 trucks, 55.7ms
Speed:

...


0: 384x640 23 cars, 1 bus, 3 trucks, 51.0ms
Speed: 3.7ms preprocess, 51.0ms inference, 2.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 23 cars, 1 bus, 3 trucks, 45.6ms
Speed: 2.8ms preprocess, 45.6ms inference, 2.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 23 cars, 3 trucks, 48.8ms
Speed: 2.9ms preprocess, 48.8ms inference, 2.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 22 cars, 2 trucks, 44.7ms
Speed: 2.5ms preprocess, 44.7ms inference, 2.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 21 cars, 3 trucks, 53.0ms
Speed: 2.6ms preprocess, 53.0ms inference, 2.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 21 cars, 3 trucks, 54.8ms
Speed: 3.9ms preprocess, 54.8ms inference, 2.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 21 cars, 3 trucks, 50.8ms
Speed: 2.7ms preprocess, 50.8ms inference, 2.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 21 cars, 2 trucks, 46.3ms
Spe

Speed: 3.6ms preprocess, 48.8ms inference, 2.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 25 cars, 1 bus, 2 trucks, 46.7ms
Speed: 2.2ms preprocess, 46.7ms inference, 2.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 25 cars, 1 bus, 2 trucks, 51.7ms
Speed: 3.5ms preprocess, 51.7ms inference, 2.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 25 cars, 1 bus, 53.3ms
Speed: 2.7ms preprocess, 53.3ms inference, 2.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 22 cars, 1 bus, 1 truck, 46.7ms
Speed: 2.2ms preprocess, 46.7ms inference, 2.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 25 cars, 1 bus, 1 truck, 48.6ms
Speed: 5.3ms preprocess, 48.6ms inference, 2.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 24 cars, 1 bus, 2 trucks, 51.1ms
Speed: 5.3ms preprocess, 51.1ms inference, 2.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 24 cars, 1 bus, 2 trucks, 47.4ms
Speed: 2.2ms prepr

Speed: 23.1ms preprocess, 265.6ms inference, 3.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 23 cars, 2 trucks, 60.9ms
Speed: 2.5ms preprocess, 60.9ms inference, 2.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 24 cars, 2 trucks, 52.2ms
Speed: 2.2ms preprocess, 52.2ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 24 cars, 2 trucks, 49.4ms
Speed: 3.4ms preprocess, 49.4ms inference, 2.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 22 cars, 2 trucks, 46.2ms
Speed: 3.7ms preprocess, 46.2ms inference, 2.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 23 cars, 2 trucks, 55.6ms
Speed: 2.2ms preprocess, 55.6ms inference, 2.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 22 cars, 2 trucks, 41.3ms
Speed: 2.8ms preprocess, 41.3ms inference, 2.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 20 cars, 2 trucks, 41.5ms
Speed: 2.7ms preprocess, 41.5ms inference, 2.4ms post


0: 384x640 22 cars, 2 trucks, 60.4ms
Speed: 2.8ms preprocess, 60.4ms inference, 3.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 22 cars, 2 trucks, 63.6ms
Speed: 3.1ms preprocess, 63.6ms inference, 2.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 21 cars, 2 trucks, 55.5ms
Speed: 2.1ms preprocess, 55.5ms inference, 2.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 20 cars, 2 trucks, 51.0ms
Speed: 3.2ms preprocess, 51.0ms inference, 2.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 20 cars, 2 trucks, 47.9ms
Speed: 2.9ms preprocess, 47.9ms inference, 2.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 22 cars, 2 trucks, 51.7ms
Speed: 3.3ms preprocess, 51.7ms inference, 2.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 22 cars, 2 trucks, 61.4ms
Speed: 3.5ms preprocess, 61.4ms inference, 2.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 21 cars, 2 trucks, 55.0ms
Speed: 3.2ms prep

113

In [158]:
cv2.waitKey(1)
cv2.destroyAllWindows()
cv2.waitKey(1)

-1